In [12]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    enableHiveSupport(). \
    config("spark.sql.warehouse.dir", f"/user/${username}/warehouse"). \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

In [13]:
spark.sql("SELECT current_database()").show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



In [15]:
spark.sql(f"USE {username}_retail")

DataFrame[]

In [16]:
spark.sql("show tables").show()

+----------------+---------+-----------+
|        database|tableName|isTemporary|
+----------------+---------+-----------+
|itversity_retail|   orders|      false|
+----------------+---------+-----------+



In [17]:
spark.sql("select * from orders").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [18]:
spark

In [19]:
spark.read

In [21]:
%%sh

ls -ltr /data/retail_db/orders/part-00000

-rw-r--r-- 1 root root 2999944 Feb 20  2017 /data/retail_db/orders/part-00000


In [22]:
%%sh

tail /data/retail_db/orders/part-00000

68874,2014-07-03 00:00:00.0,1601,COMPLETE
68875,2014-07-04 00:00:00.0,10637,ON_HOLD
68876,2014-07-06 00:00:00.0,4124,COMPLETE
68877,2014-07-07 00:00:00.0,9692,ON_HOLD
68878,2014-07-08 00:00:00.0,6753,COMPLETE
68879,2014-07-09 00:00:00.0,778,COMPLETE
68880,2014-07-13 00:00:00.0,1117,COMPLETE
68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68882,2014-07-22 00:00:00.0,10000,ON_HOLD
68883,2014-07-23 00:00:00.0,5533,COMPLETE


In [23]:
%%sh

hdfs dfs -tail /public/retail_db/orders/part-00000

014-06-12 00:00:00.0,4229,PENDING
68861,2014-06-13 00:00:00.0,3031,PENDING_PAYMENT
68862,2014-06-15 00:00:00.0,7326,PROCESSING
68863,2014-06-16 00:00:00.0,3361,CLOSED
68864,2014-06-18 00:00:00.0,9634,ON_HOLD
68865,2014-06-19 00:00:00.0,4567,SUSPECTED_FRAUD
68866,2014-06-20 00:00:00.0,3890,PENDING_PAYMENT
68867,2014-06-23 00:00:00.0,869,CANCELED
68868,2014-06-24 00:00:00.0,10184,PENDING
68869,2014-06-25 00:00:00.0,7456,PROCESSING
68870,2014-06-26 00:00:00.0,3343,COMPLETE
68871,2014-06-28 00:00:00.0,4960,PENDING
68872,2014-06-29 00:00:00.0,3354,COMPLETE
68873,2014-06-30 00:00:00.0,4545,PENDING
68874,2014-07-03 00:00:00.0,1601,COMPLETE
68875,2014-07-04 00:00:00.0,10637,ON_HOLD
68876,2014-07-06 00:00:00.0,4124,COMPLETE
68877,2014-07-07 00:00:00.0,9692,ON_HOLD
68878,2014-07-08 00:00:00.0,6753,COMPLETE
68879,2014-07-09 00:00:00.0,778,COMPLETE
68880,2014-07-13 00:00:00.0,1117,COMPLETE
68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68882,2014-07-22 00:00:00.0,10000,ON_HOLD
68883,2014-07-23 0

In [24]:
spark.read.csv?

Signature:
spark.read.csv(
    path,
    schema=None,
    sep=None,
    encoding=None,
    quote=None,
    escape=None,
    comment=None,
    header=None,
    inferSchema=None,
    ignoreLeadingWhiteSpace=None,
    ignoreTrailingWhiteSpace=None,
    nullValue=None,
    nanValue=None,
    positiveInf=None,
    negativeInf=None,
    dateFormat=None,
    timestampFormat=None,
    maxColumns=None,
    maxCharsPerColumn=None,
    maxMalformedLogPerPartition=None,
    mode=None,
    columnNameOfCorruptRecord=None,
    multiLine=None,
    charToEscapeQuoteEscaping=None,
)
Docstring:
Loads a CSV file and returns the result as a  :class:`DataFrame`.

This function will go through the input once to determine the input schema if
``inferSchema`` is enabled. To avoid going through the entire data once, disable
``inferSchema`` option or specify the schema explicitly using ``schema``.

:param path: string, or list of strings, for input path(s),
             or RDD of Strings storing CSV rows.
:param 

In [25]:
orders = spark.read.csv(
    '/public/retail_db/orders',
    schema='order_id INT, order_date STRING, order_customer_id INT, order_status STRING'
)

In [26]:
orders

DataFrame[order_id: int, order_date: string, order_customer_id: int, order_status: string]

In [28]:
orders.show(5, truncate=False)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
+--------+---------------------+-----------------+---------------+
only showing top 5 rows



In [29]:
orders.createOrReplaceTempView('orders_temp')

In [30]:
spark.sql('SHOW tables').show()

+----------------+-----------+-----------+
|        database|  tableName|isTemporary|
+----------------+-----------+-----------+
|itversity_retail|     orders|      false|
|                |orders_temp|       true|
+----------------+-----------+-----------+



In [31]:
spark.sql('SELECT count(1) FROM orders_temp').show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [33]:
spark.sql('''
    SELECT count(1) FROM orders o JOIN orders_temp ot
    ON o.order_id = ot.order_id
''').show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [35]:
spark.sql('''
INSERT OVERWRITE TABLE orders_parquet
SELECT * FROM orders_temp
''')

DataFrame[]